In [2]:
import pickle 
import pandas as pd
from sqlalchemy import create_engine, text
def insert_pred_to_sql():
    # 读取数据库连接字符串
    with open('./datasets/sql_token.pkl', 'rb') as f:
        DB_URI = pickle.load(f)

    # 创建数据库引擎
    engine = create_engine(DB_URI)

    # SQL 查询语句
    sql = text("""
        SELECT fund_code, forecast_date, pre_data, model_version, create_time, update_time
        FROM b_fund_forecast_new
        WHERE fund_code IN :codes
        ORDER BY forecast_date
    """)

    # 执行查询，传入参数（注意 tuple 中只有一个元素时加逗号）
    df = pd.read_sql_query(
        sql.bindparams(codes=tuple(['005626'])),  # 或 codes=('005626',)
        engine
    )
    
    df.to_sql('my_table', engine, if_exists='replace', index=False)
    return True

In [3]:
import pandas as pd
from sqlalchemy import create_engine, text

with open('./datasets/sql_token.pkl', 'rb') as f:
    DB_URI = pickle.load(f)

# 创建数据库引擎
engine = create_engine(DB_URI)

# SQL 查询语句
sql = text("""
    SELECT fund_code, forecast_date, pre_data, model_version, create_time, update_time
    FROM b_fund_forecast_new
    WHERE fund_code IN :codes
    ORDER BY forecast_date
""")

df = pd.read_sql_query(
    sql.bindparams(codes=tuple(['005626'])),  # 或 codes=('005626',)
    engine
)
df

,fund_code,forecast_date,pre_data,model_version,create_time,update_time
0,005626,2023-07-01,"{""pre"": [1.4995611906051636, 1.526604056358337...",V3.0,2023-08-03 17:59:16,2023-09-05 14:28:30
1,005626,2023-07-02,"{""pre"": [1.4568397998809814, 1.464197158813476...",V3.0,2023-08-04 17:07:35,2023-09-05 14:28:30
2,005626,2023-07-03,"{""pre"": [1.4621272087097168, 1.466849327087402...",V3.0,2023-08-04 19:23:31,2023-09-05 14:28:30
3,005626,2023-07-04,"{""pre"": [1.4634904861450195, 1.471401810646057...",V3.0,2023-08-07 11:30:57,2023-09-05 14:28:30
4,005626,2023-07-05,"{""pre"": [1.4883460998535156, 1.518860697746276...",V3.0,2023-08-07 17:37:31,2023-09-05 14:28:30
...,...,...,...,...,...,...
468,005626,2025-04-08,"{""pre"": [1.2187557220458984, 1.235185742378234...",V3.1,2025-04-09 00:23:01,2025-04-09 00:23:01
469,005626,2025-04-09,"{""pre"": [1.234163522720337, 1.2369379997253418...",V3.1,2025-04-10 00:23:04,2025-04-10 00:23:04
470,005626,2025-04-10,"{""pre"": [1.2268850803375244, 1.230019569396972...",V3.1,2025-04-11 00:24:03,2025-04-11 00:24:03
471,005626,2025-04-12,"{""pre"": [1.2307374477386477, 1.234757900238037...",V3.1,2025-04-13 00:23:09,2025-04-13 00:23:09


In [4]:
import os 
import torch 
import pickle 
import numpy as np
import pandas as pd 
import os 
import torch 
import pickle 
import numpy as np
import pandas as pd 


In [5]:
from run_service import *
from utils.exp_config import get_config
config = get_config('FinancialConfig')
current_date = '2025-4-12'
get_history_data(['005626'], current_date, config)

<module 'module.name' from '/home/rtx4090/code/python/当前任务/TimeSeries/configs/FinancialConfig.py'> FinancialConfig


[array([['005626', 2025, 2, 7, 4, 1.236, 1.2075, 1.047],
        ['005626', 2025, 2, 10, 0, 1.244, 1.2167, 1.055],
        ['005626', 2025, 2, 11, 1, 1.235, 1.2063, 1.046],
        ['005626', 2025, 2, 12, 2, 1.236, 1.2075, 1.047],
        ['005626', 2025, 2, 13, 3, 1.232, 1.2029, 1.043],
        ['005626', 2025, 2, 14, 4, 1.254, 1.2282, 1.065],
        ['005626', 2025, 2, 17, 0, 1.257, 1.2317, 1.068],
        ['005626', 2025, 2, 18, 1, 1.243, 1.2156, 1.054],
        ['005626', 2025, 2, 19, 2, 1.247, 1.2202, 1.058],
        ['005626', 2025, 2, 20, 3, 1.26, 1.2352, 1.071],
        ['005626', 2025, 2, 21, 4, 1.275, 1.2525, 1.086],
        ['005626', 2025, 2, 24, 0, 1.265, 1.2409, 1.076],
        ['005626', 2025, 2, 25, 1, 1.251, 1.2248, 1.062],
        ['005626', 2025, 2, 26, 2, 1.268, 1.2444, 1.079],
        ['005626', 2025, 2, 27, 3, 1.275, 1.2525, 1.086],
        ['005626', 2025, 2, 28, 4, 1.256, 1.2306, 1.067],
        ['005626', 2025, 3, 3, 0, 1.26, 1.2352, 1.071],
        ['005626',

In [6]:
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import create_engine, text
# 读取数据库连接字符串
with open('./datasets/sql_token.pkl', 'rb') as f:
    DB_URI = pickle.load(f)

# 创建数据库引擎
engine = create_engine(DB_URI)
sql = text("""
        SELECT fund_code, date, accnav, adj_nav, nav
        FROM b_fund_nav_details_new
        WHERE fund_code IN :codes
          AND date BETWEEN :start AND :end
        ORDER BY date
""")
start_date, end_date = '2021-1-1', '2025-6-19'
df = pd.read_sql_query(
    sql.bindparams(codes=tuple(['005626']), start='2021-1-1', end='2025-6-19'),
    engine
)
df

,fund_code,date,accnav,adj_nav,nav
0,005626,2021-08-17,2.062,2.0620,2.062
1,005626,2021-08-18,2.066,2.0660,2.066
2,005626,2021-08-19,2.071,2.0710,2.071
3,005626,2021-08-20,1.956,1.9560,1.956
4,005626,2021-08-23,1.979,1.9790,1.979
...,...,...,...,...,...
938,005626,2025-06-11,1.352,1.3413,1.163
939,005626,2025-06-12,1.366,1.3574,1.177
940,005626,2025-06-13,1.346,1.3344,1.157
941,005626,2025-06-16,1.338,1.3251,1.149


In [7]:
def process_date_columns(df):
    """将 date 拆成 year, month, day, weekday 四列"""
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['weekday'] = df['date'].dt.weekday
    df.drop(columns='date', inplace=True)

    cols = df.columns.tolist()
    new_order = [cols[0], 'year', 'month', 'day', 'weekday'] + [
        col for col in cols[1:] if col not in {'year', 'month', 'day', 'weekday'}
    ]
    return df[new_order].to_numpy()
df = process_date_columns(df)
df

array([['005626', 2021, 8, ..., 2.062, 2.062, 2.062],
       ['005626', 2021, 8, ..., 2.066, 2.066, 2.066],
       ['005626', 2021, 8, ..., 2.071, 2.071, 2.071],
       ...,
       ['005626', 2025, 6, ..., 1.346, 1.3344, 1.157],
       ['005626', 2025, 6, ..., 1.338, 1.3251, 1.149],
       ['005626', 2025, 6, ..., 1.319, 1.3032, 1.13]], dtype=object)

In [9]:
import pandas as pd
from datetime import datetime

def convert_array_to_df(array):
    """
    将数组转换为包含标准日期列的 DataFrame
    """
    df = pd.DataFrame(array, columns=[
        'fund_code', 'year', 'month', 'day', 'weekday',
        'value1', 'value2', 'value3'
    ])
    df['year'] = df['year'].astype(int)
    df['month'] = df['month'].astype(int)
    df['day'] = df['day'].astype(int)
    df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
    return df

def find_missing_dates_from_array_list(array_list, start_date, end_date, freq='B'):
    """
    参数：
    - array_list: List[np.ndarray]，每个元素是一个基金的历史数据数组
    - start_date, end_date: 'YYYY-MM-DD'
    - freq: 'B' 表示工作日（默认）

    返回：
    - dict: {fund_code: [缺失的日期列表]}
    """
    full_range = pd.date_range(start=start_date, end=end_date, freq=freq)
    missing_map = {}

    for arr in array_list:
        df = convert_array_to_df(arr)
        if df.empty:
            continue

        fund_code = df.iloc[0]['fund_code']
        date_series = df['date']
        missing = full_range.difference(date_series)

        if not missing.empty:
            missing_map[fund_code] = missing.strftime('%Y-%m-%d').tolist()

    return missing_map

missing_map = find_missing_dates_from_array_list([df], start_date, end_date)

for code, dates in missing_map.items():
    print(f"基金 {code} 缺失日期：{dates}")

基金 005626 缺失日期：['2021-01-01', '2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08', '2021-01-11', '2021-01-12', '2021-01-13', '2021-01-14', '2021-01-15', '2021-01-18', '2021-01-19', '2021-01-20', '2021-01-21', '2021-01-22', '2021-01-25', '2021-01-26', '2021-01-27', '2021-01-28', '2021-01-29', '2021-02-01', '2021-02-02', '2021-02-03', '2021-02-04', '2021-02-05', '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-11', '2021-02-12', '2021-02-15', '2021-02-16', '2021-02-17', '2021-02-18', '2021-02-19', '2021-02-22', '2021-02-23', '2021-02-24', '2021-02-25', '2021-02-26', '2021-03-01', '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05', '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11', '2021-03-12', '2021-03-15', '2021-03-16', '2021-03-17', '2021-03-18', '2021-03-19', '2021-03-22', '2021-03-23', '2021-03-24', '2021-03-25', '2021-03-26', '2021-03-29', '2021-03-30', '2021-03-31', '2021-04-01', '2021-04-02', '2021-04-05', '2021-04-06', '2021-04-07', '2021-04-08', '202

In [18]:
import numpy as np
from scipy.interpolate import interp1d

def cubic_spline_interpolate_nan(y):
    """
    对一维数组中的 np.nan 位置进行三次样条插值。
    
    参数:
        y: np.ndarray，包含缺失值（np.nan）的目标数组
        x: 可选，np.ndarray，表示横轴坐标（默认等间距，即 np.arange(len(y))）
    
    返回:
        y_interp: np.ndarray，插值填充后的数组
    """
    y = np.asarray(y)
    x = np.arange(len(y))

    mask = ~np.isnan(y)
    if np.sum(mask) < 4:
        raise ValueError("进行三次样条插值至少需要4个非缺失点")

    f_interp = interp1d(x[mask], y[mask], kind='cubic', fill_value="extrapolate")

    y_filled = y.copy()
    y_filled[np.isnan(y)] = f_interp(x[np.isnan(y)])
    return y_filled

import matplotlib.pyplot as plt

y = np.array([1.0, 2.1, np.nan, 3.9, 5.2, np.nan, 6.8, 7.9, np.nan, 9.9])
y_interp = cubic_spline_interpolate_nan(y)

array([1.        , 2.1       , 2.91475048, 3.9       , 5.2       ,
       6.0597877 , 6.8       , 7.9       , 9.08695892, 9.9       ])

In [15]:
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import create_engine, text
# 读取数据库连接字符串
with open('./datasets/sql_token.pkl', 'rb') as f:
    DB_URI = pickle.load(f)

# 创建数据库引擎
engine = create_engine(DB_URI)
sql = text("""
        SELECT fund_code, date, nav, accnav, adj_nav
        FROM b_fund_nav_details_new
        WHERE fund_code IN :codes
          AND date BETWEEN :start AND :end
        ORDER BY date
""")
df = pd.read_sql_query(
    sql.bindparams(codes=tuple(['005626']), start='2021-1-1', end='2025-6-19'),
    engine
)
df

,fund_code,date,nav,accnav,adj_nav
0,005626,2021-08-17,2.062,2.062,2.0620
1,005626,2021-08-18,2.066,2.066,2.0660
2,005626,2021-08-19,2.071,2.071,2.0710
3,005626,2021-08-20,1.956,1.956,1.9560
4,005626,2021-08-23,1.979,1.979,1.9790
...,...,...,...,...,...
938,005626,2025-06-11,1.163,1.352,1.3413
939,005626,2025-06-12,1.177,1.366,1.3574
940,005626,2025-06-13,1.157,1.346,1.3344
941,005626,2025-06-16,1.149,1.338,1.3251
